<a href="https://colab.research.google.com/github/ncfausti/Data-Science/blob/master/sonogram-to-ct-scan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd drive/MyDrive

/content/drive/MyDrive


In [3]:
!pwd

/content/drive/MyDrive


In [22]:
MY_DRIVE = '/content/drive/MyDrive/'

X_TRAIN_PATH = MY_DRIVE + "Overview-Ind-Study/Overview/data/X_train/"
Y_TRAIN_PATH = MY_DRIVE + "Overview-Ind-Study/Overview/data/y_train/"

# find the number of filenames that are in both, use this value as first 
# dimension of training/test tensors, i.e. number of training instances
CMD_RESULT = !find $X_TRAIN_PATH $Y_TRAIN_PATH -printf '%P\n' | sort | uniq -d | wc -l
NUM_TRAINING_INSTANCES = int(CMD_RESULT[0])

print(NUM_TRAINING_INSTANCES)

assert NUM_TRAINING_INSTANCES > 500

2420


In [ ]:
import numpy as np
import os
from PIL import Image
from keras.layers.merge import add
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras import backend as K
from keras.layers.convolutional import Convolution2D
import six

from keras.models import Model
from keras.layers import (
    Input,
    Activation,
    Dense,
    Flatten
)

from keras.layers.convolutional import (
    Conv2D,
    MaxPooling2D,
    AveragePooling2D
)

import os.path
from os import path

ROW_AXIS = 1
COL_AXIS = 2
CHANNEL_AXIS = 3

MY_DRIVE = '/content/drive/MyDrive/'

X_TRAIN_PATH = MY_DRIVE + "Overview-Ind-Study/Overview/data/X_train/"
Y_TRAIN_PATH = MY_DRIVE + "Overview-Ind-Study/Overview/data/y_train/"


#########  https://www.programcreek.com/python/example/101438/keras.layers.merge.add #################


def extract_data(xPath, yPath):
#   drive.mount('/content/drive')
  %cd "/drive/MyDrive" 
  #!ls
  
  # using all 2420 images now, instead of 1000 bc we have extra RAM
  X, y = np.zeros((,512,512,3)), np.zeros((1000,512,512,3)) 
  
  index = 0
  for file in range(0, 2424):

    filename_x = xPath + str(file)+'.jpg'
    filename_y = yPath + str(file)+'.jpg'

    if path.exists(filename_x) and path.exists(filename_y):
        img = Image.open(filename_x).convert('RGB')
        if file % 100 == 0:
            print("extracting... ", file," of 1000")
        imgArray = np.asarray(img, dtype='uint8')
        imgArray =  imgArray / 255.0
        X[index] = imgArray

        img = Image.open(filename_y).convert('RGB')
        imgArray = np.asarray(img, dtype='uint8')
        imgArray =  imgArray / 255.0
        y[index] = imgArray
        
        index += 1
  return X, y


def _shortcut(input, residual):
    """Adds a shortcut between input and residual block and merges them with "sum"
    """
    # Expand channels of shortcut to match residual.
    # Stride appropriately to match residual (width, height)
    # Should be int if network architecture is correctly configured.
    input_shape = K.int_shape(input)
    residual_shape = K.int_shape(residual)
    stride_width = int(round(input_shape[ROW_AXIS] / residual_shape[ROW_AXIS]))
    stride_height = int(round(input_shape[COL_AXIS] / residual_shape[COL_AXIS]))
    equal_channels = input_shape[CHANNEL_AXIS] == residual_shape[CHANNEL_AXIS]

    shortcut = input
    # 1 X 1 conv if shape is different. Else identity.
    if stride_width > 1 or stride_height > 1 or not equal_channels:
        shortcut = Conv2D(filters=residual_shape[CHANNEL_AXIS],
                          kernel_size=(1, 1),
                          strides=(stride_width, stride_height),
                          padding="valid",
                          kernel_initializer="he_normal",
                          kernel_regularizer=l2(0.0001))(input)

    return add([shortcut, residual])

In [ ]:
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import Convolution1D
from keras.layers import Conv2DTranspose ## also try UpSampling2D
from keras.layers import UpSampling2D
from keras.layers import MaxPooling2D

from keras.callbacks import ModelCheckpoint
import keras

import os.path
from os import path

from keras.layers import merge, Convolution2D, Input

X_TEST_PATH = MY_DRIVE + "Overview-Ind-Study/Overview/data/X_test/"
Y_TEST_PATH = MY_DRIVE + "Overview-Ind-Study/Overview/data/y_test/"

def generate_model():  
  poolsize = 2 # (2,2)
  kerelsize = 3 # (3,3)
  x = Input(shape=(512,512,3))
  
  a = Convolution2D(3, kerelsize, activation="relu", padding="same")(x) ### save copy for later, use in upsampling (resnet/symmetric skip connection)
  
  b = MaxPooling2D(pool_size=poolsize, strides=2)(a)
  
  c = Convolution2D(3, kerelsize, activation="relu", padding="same")(b)
  
  d = MaxPooling2D(pool_size=poolsize, strides=2)(c)
    
  e = Convolution2D(3, kerelsize, activation="relu", padding="same")(d)
  
  f = MaxPooling2D(pool_size=poolsize, strides=2)(e)
  
  g = Convolution2D(3, kerelsize, activation="relu", padding="same")(f)
  
  h = MaxPooling2D(pool_size=poolsize, strides=2)(g)
  
  #----
  i = Convolution2D(3, kerelsize, activation="relu", padding="same")(h)
  # ---
  
  j = UpSampling2D(size=poolsize, interpolation='nearest')(i)
  j1 = _shortcut(g, j) # combine the saved convolutional layer and upsampling
  
  k = Convolution2D(3, kerelsize, activation="relu", padding="same")(j1)
  
  l = UpSampling2D(size=poolsize, interpolation='nearest')(k)
  l1 = _shortcut(e, l)
  
  m = Convolution2D(3, kerelsize, activation="relu", padding="same")(l1) 
  
  n = UpSampling2D(size=poolsize, interpolation='nearest')(m)
  l1 = _shortcut(c, n)
  
  o = Convolution2D(3, kerelsize, activation="relu", padding="same")(n) 
  
  p = UpSampling2D(size=poolsize, interpolation='nearest')(o)
  p1 = _shortcut(a, p)
  
  q = Convolution2D(3, kerelsize, activation="relu", padding="same")(p1)
  
  r = Convolution2D(3, poolsize, activation="linear", padding="same")(q)
  
  model = Model(inputs=x, outputs = r)
  
  adam = keras.optimizers.Adam(lr=0.0001)
  
  model.compile(optimizer=adam, loss="mse", metrics=['accuracy'])
  return model


def extract_data_testing(xPath, yPath):
#   drive.mount('/content/drive')
#   %cd "/content/drive/MyDrive" 
  #!ls
  
  X, y = np.zeros((12,512,512,3)), np.zeros((12,512,512,3))
  
  index = 0
  for file in os.listdir(xPath):
    if path.exists(xPath + file):
        img = Image.open(xPath + file).convert('RGB')
        imgArray = np.asarray(img, dtype='uint8')
        imgArray =  imgArray / 255.0
        X[index] = imgArray

        img = Image.open(yPath + file).convert('RGB')
        imgArray = np.asarray(img, dtype='uint8')
        imgArray =  imgArray / 255.0
        y[index] = imgArray
        
        index += 1
  return X, y


In [ ]:
# Setup the model
model = generate_model()

In [ ]:
!echo $X_TRAIN_PATH
!ls {X_TRAIN_PATH}/*.jpg | wc -l
!ls  {X_TRAIN_PATH}/5.jpg 

/content/drive/MyDrive/Overview-Ind-Study/Overview/data/X_train/
2419
ls: cannot access '/content/drive/MyDrive/Overview-Ind-Study/Overview/data/X_train//5.jpg': No such file or directory


In [ ]:
# Extract training data (memory intensive)
X, y = extract_data(X_TRAIN_PATH, Y_TRAIN_PATH)

[Errno 2] No such file or directory: '/drive/MyDrive'
/content/drive/MyDrive
extracting...  0  of 1000
extracting...  100  of 1000
extracting...  200  of 1000
extracting...  300  of 1000
extracting...  400  of 1000
extracting...  500  of 1000
extracting...  600  of 1000
extracting...  700  of 1000
extracting...  800  of 1000
extracting...  900  of 1000
extracting...  1000  of 1000


IndexError: ignored

In [ ]:
# Save a dictionary into a pickle file.
import pickle
import os.path

file_path = "pkl.pkl"
n_bytes = 2**31
max_bytes = 2**31 - 1
data = bytearray(n_bytes)

## write
bytes_out = pickle.dumps(data)
with open(file_path, 'wb') as f_out:
    for idx in range(0, len(bytes_out), max_bytes):
        f_out.write(bytes_out[idx:idx+max_bytes])

## read
bytes_in = bytearray(0)
input_size = os.path.getsize(file_path)
with open(file_path, 'rb') as f_in:
    for _ in range(0, input_size, max_bytes):
        bytes_in += f_in.read(max_bytes)
data2 = pickle.loads(bytes_in)

assert(data == data2)

In [ ]:
len(x)

NameError: ignored

In [ ]:
extracted = { "X": X, "y": y }
pickle.dumps( extracted, open( "dec-15-baseline.pkl", "wb" ) )


NameError: ignored

In [ ]:
# Fit the model
history = model.fit(X, y, batch_size=16, epochs=50, validation_split=0.2, shuffle=True)

Epoch 1/50
50/50 [==============================] - 46s 921ms/step - loss: 0.0771 - accuracy: 0.0495 - val_loss: 0.0722 - val_accuracy: 0.0126
Epoch 2/50
50/50 [==============================] - 45s 896ms/step - loss: 0.0578 - accuracy: 0.0039 - val_loss: 0.0546 - val_accuracy: 0.0024
Epoch 3/50
50/50 [==============================] - 45s 898ms/step - loss: 0.0478 - accuracy: 0.0023 - val_loss: 0.0465 - val_accuracy: 0.0027
Epoch 4/50
50/50 [==============================] - 45s 900ms/step - loss: 0.0421 - accuracy: 0.0034 - val_loss: 0.0410 - val_accuracy: 0.0052
Epoch 5/50
50/50 [==============================] - 45s 907ms/step - loss: 0.0380 - accuracy: 0.0075 - val_loss: 0.0367 - val_accuracy: 0.0136
Epoch 6/50
50/50 [==============================] - 45s 910ms/step - loss: 0.0346 - accuracy: 0.0183 - val_loss: 0.0329 - val_accuracy: 0.0307
Epoch 7/50
50/50 [==============================] - 45s 902ms/step - loss: 0.0315 - accuracy: 0.0391 - val_loss: 0.0294 - val_accuracy: 0.0652

In [ ]:



model.save("Overview-Ind-Study/Overview/model.h5")

# evaluate model
X_test, y_test = extract_data_testing(X_TEST_PATH, Y_TEST_PATH)
score = model.evaluate(X_test, y_test)
print("scoring ...")
print (model.metrics_names, score)

# make prediction  n
out_array = model.predict(X_test)

out_array = out_array.astype(np.float32) * 255. 
out_array = np.clip(out_array, 0, 255).astype('uint8') 

num, rows, cols, channels = out_array.shape

for n in range(num):
    filename= str(n) + ".jpg"
    # save prediction
    out_img = Image.fromarray(out_array[n]).convert('L') # convert to black and white
    out_img.save("Overview-Ind-Study/Overview/out_" + filename, format="JPEG")
    out_img.show()


NameError: ignored

In [ ]:
import matplotlib.pyplot as plt
# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
OVERVIEW_DIR = '/content/drive/MyDrive/Overview-Ind-Study/Overview/'
from tensorflow import keras
model = keras.models.load_model(OVERVIEW_DIR + 'model.h5')

In [ ]:
history = model.fit(X, y, batch_size=16, epochs=50, validation_split=0.2, shuffle=True)
print("model fit ...")
model.save("Overview-Ind-Study/Overview/model.h5")

# evaluate model
X_test, y_test = extract_data_testing(X_TEST_PATH, Y_TEST_PATH)
score = model.evaluate(X_test, y_test)
print("scoring ...")
print (model.metrics_names, score)

# make prediction  n
out_array = model.predict(X_test)

out_array = out_array.astype(np.float32) * 255. 
out_array = np.clip(out_array, 0, 255).astype('uint8') 

num, rows, cols, channels = out_array.shape
for n in range(num):
    filename= str(n) + ".jpg"
# save prediction
out_img = Image.fromarray(out_array[n]).convert('L') # convert to black and white
out_img.save("Overview-Ind-Study/Overview/out_" + filename, format="JPEG")
out_img.show()


import matplotlib.pyplot as plt
# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

NameError: ignored

In [ ]:
# train model
  model = generate_model()
  print("model generated ...")       
  X, y = extract_data(X_TRAIN_PATH, Y_TRAIN_PATH)
  print("data extracted ...")
  history = model.fit(X, y, batch_size=16, epochs=50, validation_split=0.2, shuffle=True)
  print("model fit ...")
  model.save("Overview-Ind-Study/Overview/model.h5")
  
  # evaluate model
  X_test, y_test = extract_data_testing(X_TEST_PATH, Y_TEST_PATH)
  score = model.evaluate(X_test, y_test)
  print("scoring ...")
  print (model.metrics_names, score)
  
  # make prediction  n
  out_array = model.predict(X_test)

  out_array = out_array.astype(np.float32) * 255. 
  out_array = np.clip(out_array, 0, 255).astype('uint8') 

  num, rows, cols, channels = out_array.shape
  for n in range(num):
    filename= str(n) + ".jpg"
    # save prediction
    out_img = Image.fromarray(out_array[n]).convert('L') # convert to black and white
    out_img.save("Overview-Ind-Study/Overview/out_" + filename, format="JPEG")
    out_img.show()
    
    
  import matplotlib.pyplot as plt
  # Plot training & validation accuracy values
  plt.plot(history.history['acc'])
  plt.plot(history.history['val_acc'])
  plt.title('Model accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Test'], loc='upper left')
  plt.show()

  # Plot training & validation loss values
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('Model loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Test'], loc='upper left')
  plt.show()

NameError: ignored